# Stacking

In [58]:
!pip3 install scikit-learn

### Importing Libraries

In [45]:
#importing necessary libraries
import numpy as np
import pandas as pd

### Importing Dataset

In [46]:
data = pd.read_csv('data_cleaned.csv')

#printing the first few rows
data.head()

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,SibSp_0,SibSp_1,...,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6,Embarked_C,Embarked_Q,Embarked_S
0,0,22.0,7.2500,0,0,1,0,1,0,1,...,1,0,0,0,0,0,0,0,0,1
1,1,38.0,71.2833,1,0,0,1,0,0,1,...,1,0,0,0,0,0,0,1,0,0
2,1,26.0,7.9250,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
3,1,35.0,53.1000,1,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,1
4,0,35.0,8.0500,0,0,1,0,1,1,0,...,1,0,0,0,0,0,0,0,0,1


In [47]:
#size of the data
data.shape

(891, 25)

In [48]:
#checking missing values
data.isnull().sum()

Survived      0
Age           0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Sex_female    0
Sex_male      0
SibSp_0       0
SibSp_1       0
SibSp_2       0
SibSp_3       0
SibSp_4       0
SibSp_5       0
SibSp_8       0
Parch_0       0
Parch_1       0
Parch_2       0
Parch_3       0
Parch_4       0
Parch_5       0
Parch_6       0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

### Separating Dependent and Independent Variables

In [49]:
#independent variables
x = data.drop(["Survived"], axis = 1)

#dependent variables
y = data['Survived']

x.shape, y.shape

((891, 24), (891,))

### Making test and training set

In [50]:
from sklearn.model_selection import train_test_split as tts

train_x, test_x, train_y, test_y = tts (x, y, random_state = 9 , stratify = y)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((668, 24), (223, 24), (668,), (223,))

## Base models 

In [51]:
#importing predictive models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

<img src="stacking image.png" alt="Drawing" style="width: 400px;"/>

### Model Training and Predictions 

In [52]:
def model_predictions(model, train_x, train_y, test_x):
    
    #train the model
    model.fit(train_x,train_y)
    
    #storing predictions for train and test
    pred_train=model.predict(train_x)
    pred_test=model.predict(test_x)
    return pred_train, pred_test

In [53]:
#Model 1 - Decision Tree
DT=DecisionTreeClassifier(random_state= 101)
M1_train, M1_test = model_predictions(DT, train_x, train_y, test_x)

In [54]:
#Feature Scaling
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# scaler.fit(X=train_x)

# train_x = scaler.transform(train_x)
# test_x = scaler.transform(test_x)

In [55]:
#Model 2 - Logistic Regression
LR=LogisticRegression(random_state= 101)
M2_train, M2_test = model_predictions(LR, train_x, train_y, test_x)

C:\Users\AhmedSahal\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [56]:
#Model 3 - k Nearest Neighbour
model=KNeighborsClassifier(n_neighbors=9)
model.fit(train_x,train_y)
    
#storing predictions for train and test
pred_train =model.predict(train_x)
pred_test=model.predict(test_x)
# return pred_train, pred_test
# M3_train, M3_test = model_predictions(knn, train_x, train_y, test_x)/

AttributeError: 'NoneType' object has no attribute 'split'

## Stacking Model

<img src="stacking image 2.png" alt="Drawing" style="width: 500px;"/>

In [13]:
#Creating a New train dataframe
train_prediction = {
              'DT': M1_train,
              'LR': M2_train,
              'knn': M3_train
              }
train_predictions = pd.DataFrame(train_prediction)
train_predictions.head()

,DT,LR,knn
0,1,1,0
1,1,1,1
2,1,1,0
3,0,1,1
4,0,0,0


In [14]:
#Creating a New test dataframe
test_prediction = {
              'DT': M1_test,
              'LR': M2_test,
              'knn': M3_test
              }
test_predictions = pd.DataFrame(test_prediction)
test_predictions.head()

,DT,LR,knn
0,0,0,0
1,1,1,1
2,0,0,0
3,1,0,0
4,0,0,0


In [15]:
# Stacker Model
model = LogisticRegression()
model.fit(train_predictions, train_y)
model.score(test_predictions,test_y)

/home/aishwarya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.726457399103139